Demo for a minor performance issue. See https://github.com/JiaweiZhuang/xESMF/issues/11#issuecomment-416397416

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from scipy.sparse import coo_matrix

# Preparation

In [2]:
ds = xr.open_dataset("weights.nc")
n_s = ds.dims['n_s']
col = ds['col'].values - 1
row = ds['row'].values - 1
S = ds['S'].values

In [3]:
A = coo_matrix((S, (row, col))) 
A.shape

(120000, 240000)

In [4]:
data = np.random.rand(500, A.shape[1])
data.shape

(500, 240000)

# Standard layout

In [5]:
%timeit A.dot(data.T).T

598 ms ± 25.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Transposed layout

In [6]:
# emulate the return of regrid.get_factors()
index_list = np.c_[row, col].T
index_list.shape

(2, 480000)

In [7]:
index_list.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [8]:
index_list[0,:].flags # the row indice are not contiguous

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [9]:
A_trans = coo_matrix((S, (index_list[0,:], index_list[1,:]))) 
A_trans.shape

(120000, 240000)

In [10]:
A_trans.row.flags  # the row indice are not contiguous

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [11]:
%timeit A_trans.dot(data.T).T # take longer than the standard A

871 ms ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
